In [6]:
# Nếu chạy trong Jupyter, thêm project root vào sys.path
import sys, os, logging

# CHỈNH cho đúng với máy bạn:
PROJECT_ROOT = r"E:\Job\Kin-Hotel\DE\KinHotelAutoDashboard"
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Log mức INFO cho gọn
logging.getLogger().setLevel("INFO")


In [7]:
import asyncio

# Nếu notebook báo "event loop is running", bạn có thể dùng "await func()" thay vì asyncio.run(func()).
# Hoặc cài nest_asyncio: pip install nest_asyncio
# import nest_asyncio; nest_asyncio.apply()


In [8]:
from datetime import datetime, timedelta, timezone

from src.data_pipeline.extractors.pms.booking import BookingListExtractor
from src.utils.date_params import DateWindow, ICT

async def smoke_test_bookings(
    branch_ids=None,
    hours=24,
    limit=15,                 # để ép phân trang nếu dữ liệu nhiều
    field="check_in",         # "check_in" hoặc "create"
    max_concurrent=3
):
    """
    Extract bookings từ PMS cho nhiều branch song song trong khoảng {hours} giờ gần nhất.
    - Không transform, không load
    - In ra số record và 1-2 record mẫu
    """
    ex = BookingListExtractor()

    end_utc = datetime.now(timezone.utc)
    start_utc = end_utc - timedelta(hours=hours)

    # Khung thời gian chuẩn hoá → PMS params (DateWindow sẽ format theo ICT)
    dw_template = DateWindow.from_utc(start_utc, end_utc, field=field, tz=ICT)

    # Nếu không truyền, lấy toàn bộ branch có trong map
    if branch_ids is None:
        branch_ids = list(BookingListExtractor.TOKEN_BRANCH_MAP.keys())

    sem = asyncio.Semaphore(max_concurrent)

    async def run_one(bid: int):
        async with sem:
            # Tạo DateWindow riêng cho branch (tránh mutate)
            dw = DateWindow(start=dw_template.start, end=dw_template.end, field=field, tz=ICT)
            # Gọi extract_async trực tiếp, truyền limit để dễ thấy phân trang
            res = await ex.extract_async(branch_id=bid, date_window=dw, limit=limit)
            return bid, res

    results = await asyncio.gather(*[run_one(b) for b in branch_ids], return_exceptions=True)

    # Đóng session HTTP
    await ex.close()

    # Tổng kết
    ok = 0
    total = 0
    for item in results:
        if isinstance(item, Exception):
            print("Branch task error:", item)
            continue
        bid, res = item
        total += 1
        name = BookingListExtractor.TOKEN_BRANCH_MAP.get(bid, f"Branch {bid}")
        if res.is_success:
            ok += 1
            print(f"[OK] {name} (id={bid}) → {res.record_count} records")
            if res.record_count:
                # In thử 1-2 record đầu cho biết schema
                print("  sample:", res.data[:2])
        else:
            print(f"[FAIL] {name} (id={bid}) → {res.error}")

    print(f"\nDONE: {ok}/{total} branches OK")


In [ ]:
# Ví dụ chạy 3 branch cho nhanh
test_branches = [1, 2,3,4,5, 6,7,,9,10]   # đổi theo nhu cầu

# Nếu cell hỗ trợ await:
await smoke_test_bookings(
    branch_ids=test_branches,
    hours=24,
    limit=15,
    field="check_in",      # hoặc "create" nếu muốn test theo ngày tạo
    max_concurrent=2
)

# Nếu không thể dùng await, thử:
# asyncio.run(smoke_test_bookings(branch_ids=test_branches, hours=24, limit=15, field="check_in", max_concurrent=2))


[2025-08-31 16:47:19] [env_utils.py] [INFO] 🔍 Checking environment variables...
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ ALGORITHM: HS256 [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ ODOO_PASSWORD: ***HIDDEN*** [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ EMAIL_SENDER: thanh... [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ ODOO_EMAIL: thanh... [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ ODOO_URL: https... [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ PMS_PASSWORD: ***HIDDEN*** [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ EMAIL_PASSWORD: ***HIDDEN*** [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ MAX_WORKERS: 5 [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ EMAIL_RECIPIENT: thanh... [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ RETRY_LIMIT: 3 [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ PMS_BASE_URL: https... [PASS]
[2025-08-31 16:47:19] [env_utils.py] [INFO] ✅ RETRY_DELAY: 5 [PASS]
[2025-08-31 16:47:19] [env